In [217]:
import pandas as pd
import numpy as np


df0 = pd.read_csv('../lab_01/perfume/data.csv', delimiter=',', encoding="utf8")
df = df0.copy(deep=True)
df

,Название,Пол,Бренд,Страна,Сегмент,Концентрация,Верхние ноты,Ноты сердца,Базовые ноты,Запахи,...,Шлейф,Время года,Возраст (до 18),Возраст (до 26),Возраст (до 35),Возраст (до 45),Возраст (после 45),мл,Цена,Характеристики аромата
0,Shalimar,ж,Guerlain,Франция,люкс,духи,Бергамот;Цитрусы;Лимон;Мандарин,Ирис;Пачули;Роза;Ветивер;Жасмин,Бобы;Кожаный аккорд;Ладан;Мускус;Опопанакс;Сан...,NaN,...,средний,зима;осень,+,+,+,+,+,30.0,8183,NaN
1,3 L'Imperatrice,ж,Dolce&Gabbana,Италия,люкс,туалетная вода,Киви;Ревень;Розовый перец,Арбуз;Цикламен;Жасмин,Мускус;Сандал,NaN,...,очень заметный,весна;лето,+,+,+,+,+,50.0,7949,Арбуз;Тропические фрукты;Повседневный;Свежий
2,1001 Nights,ж,Ajmal,ОАЭ,люкс,масляные духи,NaN,NaN,NaN,Древесный аккорд;Мускус;Специи,...,средний,NaN,+,+,+,+,+,60.0,2000,NaN
3,Sabina,ж,Arabesque Perfumes,ОАЭ,нишевая парфюмерия,духи,NaN,NaN,NaN,Ананас;Апельсин;Мускус;Киви;Сахар;Тиаре;Ваниль...,...,средний,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3600,NaN
4,Resolute Gold,м,Khalis,ОАЭ,масс-маркет,парфюмерная вода,Белые цветы;Клубника,Мускатный орех;Жасмин,Амбра;Древесный аккорд;Мускус;Сандал,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,1802,NaN
5,Oath for Him,м,Ajmal,ОАЭ,люкс,парфюмерная вода,Апельсин;Бергамот;Черная смородина;Дыня;Яблоко...,Герань;Ирис;Лаванда;Роза;Жасмин,Амбра;Кедр;Мох;Мускус;Пачули;Ветивер,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,9699,NaN
6,Twist,ж,Emper,ОАЭ,масс-маркет,парфюмерная вода,Бергамот;Чёрная смородина,Фиалка;Роза;Жасмин,Мускус;Ваниль,NaN,...,заметный,зима;весна,+,+,+,NaN,NaN,100.0,1872,NaN
7,CK Everyone,у,Calvin Klein,США,люкс,туалетная вода,Апельсин,Чай,Кедр;Мускус,NaN,...,средний,весна;лето,NaN,+,+,NaN,NaN,50.0,4749,NaN
8,Bad Boy,м,Carolina Herrera,США,люкс,туалетная вода,Белый перец;Бергамот;Чёрный перец,Белый кедр;Шалфей,Бобы;Янтарь;Какао,NaN,...,средний,зима;осень,NaN,+,+,NaN,NaN,100.0,10899,NaN
9,Eau de Lacoste L.12.12 Blanc,м,Lacoste,Франция,люкс,туалетная вода,Грейпфрут;Кардамон;Розмарин,Иланг-иланг;Тубероза,Кедр;Кожаный аккорд;Ветивер;Замша,NaN,...,средний,лето;весна,+,NaN,NaN,NaN,NaN,50.0,8549,Грейпфрут;Кожа;Спортивный;Свежий;Повседневный


In [218]:
smells_df = pd.read_csv('../lab_01/perfume/smells.csv', delimiter=',', encoding="utf8")
smells_df

,Семейство,Ассоциации,Характеристики
0,Восточные,шоколад;карамель;дым;благовоние;смола;пряность...,плотный;насыщенный;торжественный;чувственный;т...
1,Древесные,сандал;мох;папоротник;древесина;смола;дым;земл...,теплый;сливочный;сладкий;чувственный;согревающ...
2,Цитрусовые,апельсин;лайм;грейпфрут;лимон;бергамот;виногра...,легкий;свежий;бодрящий;энергичный;жизнерадостн...
3,Фруктовые,NaN,сладкий;свежий;игривый;необычный
4,Свежие,горный воздух;морозная свежесть;огурец;зеленое...,тонкий;свежий;ненавязчивый;морской;спокойный
5,Шипровые,мох;бергамот,чувственный;согревающий;таинственный
6,Пряные,перец;корица;гвоздика;куркума;имбирь;орех,согревающий;активный;острый
7,Цветочные,NaN,сладкий;массивный;свежий;легкий
8,Фужерные,папоротник;лаванда;герань;мох;бергамот;сухая т...,терпкий;пряно-горький
9,Мускусные,NaN,теплый;мягкий;пряный;древесный;телесный;ненавя...


In [219]:
from cmath import isnan, nan


df["Пол"], _ = pd.factorize(df["Пол"])
df["Бренд"], _ = pd.factorize(df["Бренд"])
df["Страна"], _ = pd.factorize(df["Страна"])
df["Сегмент"], _ = pd.factorize(df["Сегмент"])
df["Концентрация"], _ = pd.factorize(df["Концентрация"])

df["Верхние ноты"] = df["Верхние ноты"].map(lambda elem: str(elem).split(';'))
df["Ноты сердца"] = df["Ноты сердца"].map(lambda elem: str(elem).split(';'))
df["Базовые ноты"] = df["Базовые ноты"].map(lambda elem: str(elem).split(';'))
df["Запахи"] = df["Запахи"].map(lambda elem: str(elem).split(';'))
df["Семейства"] = df["Семейства"].map(lambda elem: str(elem).split(';'))
df["Время года"] = df["Время года"].map(lambda elem: str(elem).split(';'))

df["Стойкость минимум"] = df["Стойкость минимум"].values / max(df["Стойкость минимум"].values)
df["Стойкость максимум"] = df["Стойкость максимум"].values / max(df["Стойкость максимум"].values)

df["Шлейф"], _ = pd.factorize(df["Шлейф"])

df["Возраст (до 18)"] = df["Возраст (до 18)"].map(lambda elem: 1 if elem == '+' else 0)
df["Возраст (до 26)"] = df["Возраст (до 26)"].map(lambda elem: 1 if elem == '+' else 0)
df["Возраст (до 35)"] = df["Возраст (до 35)"].map(lambda elem: 1 if elem == '+' else 0)
df["Возраст (до 45)"] = df["Возраст (до 45)"].map(lambda elem: 1 if elem == '+' else 0)
df["Возраст (после 45)"] = df["Возраст (после 45)"].map(lambda elem: 1 if elem == '+' else 0)
df["Характеристики аромата"] = df["Характеристики аромата"].map(lambda elem: str(elem).split(';'))

df["Цена за 1 мл"] = df["Цена"] / df["мл"]


del df["мл"]
del df["Цена"]



df

,Название,Пол,Бренд,Страна,Сегмент,Концентрация,Верхние ноты,Ноты сердца,Базовые ноты,Запахи,...,Стойкость максимум,Шлейф,Время года,Возраст (до 18),Возраст (до 26),Возраст (до 35),Возраст (до 45),Возраст (после 45),Характеристики аромата,Цена за 1 мл
0,Shalimar,0,0,0,0,0,"[Бергамот, Цитрусы, Лимон, Мандарин]","[Ирис, Пачули, Роза, Ветивер, Жасмин]","[Бобы, Кожаный аккорд, Ладан, Мускус, Опопанак...",[nan],...,0.416667,0,"[зима, осень]",1,1,1,1,1,[nan],272.766667
1,3 L'Imperatrice,0,1,1,0,1,"[Киви, Ревень, Розовый перец]","[Арбуз, Цикламен, Жасмин]","[Мускус, Сандал]",[nan],...,NaN,1,"[весна, лето]",1,1,1,1,1,"[Арбуз, Тропические фрукты, Повседневный, Свежий]",158.980000
2,1001 Nights,0,2,2,0,2,[nan],[nan],[nan],"[Древесный аккорд, Мускус, Специи]",...,1.000000,0,[nan],1,1,1,1,1,[nan],33.333333
3,Sabina,0,3,2,1,0,[nan],[nan],[nan],"[Ананас, Апельсин, Мускус, Киви, Сахар, Тиаре,...",...,1.000000,0,[nan],0,0,0,0,0,[nan],600.000000
4,Resolute Gold,1,4,2,2,3,"[Белые цветы, Клубника]","[Мускатный орех, Жасмин]","[Амбра, Древесный аккорд, Мускус, Сандал]",[nan],...,NaN,-1,[nan],0,0,0,0,0,[nan],18.020000
5,Oath for Him,1,2,2,0,3,"[Апельсин, Бергамот, Черная смородина, Дыня, Я...","[Герань, Ирис, Лаванда, Роза, Жасмин]","[Амбра, Кедр, Мох, Мускус, Пачули, Ветивер]",[nan],...,NaN,-1,[nan],0,0,0,0,0,[nan],193.980000
6,Twist,0,5,2,2,3,"[Бергамот, Чёрная смородина]","[Фиалка, Роза, Жасмин]","[Мускус, Ваниль]",[nan],...,1.000000,2,"[зима, весна]",1,1,1,0,0,[nan],18.720000
7,CK Everyone,2,6,3,0,1,[Апельсин],[Чай],"[Кедр, Мускус]",[nan],...,0.416667,0,"[весна, лето]",0,1,1,0,0,[nan],94.980000
8,Bad Boy,1,7,3,0,1,"[Белый перец, Бергамот, Чёрный перец]","[Белый кедр, Шалфей]","[Бобы, Янтарь, Какао]",[nan],...,0.416667,0,"[зима, осень]",0,1,1,0,0,[nan],108.990000
9,Eau de Lacoste L.12.12 Blanc,1,8,0,0,1,"[Грейпфрут, Кардамон, Розмарин]","[Иланг-иланг, Тубероза]","[Кедр, Кожаный аккорд, Ветивер, Замша]",[nan],...,0.416667,0,"[лето, весна]",1,0,0,0,0,"[Грейпфрут, Кожа, Спортивный, Свежий, Повседне...",170.980000


In [220]:
print()